# COMPUTE EIGENFACES 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams.update({'font.size': 18})

mat_contents = scipy.io.loadmat(os.path.join('..','DATA','allFaces.mat'))
faces = mat_contents['faces']
m = int(mat_contents['m'])
n = int(mat_contents['n'])
nfaces = np.ndarray.flatten(mat_contents['nfaces'])

# We use the first 36 people for training data
trainingFaces = faces[:,:np.sum(nfaces[:36])]
print(trainingFaces.shape) # 모든 얼굴

avgFace = np.mean(trainingFaces,axis=1) # size n*m by 1
print(avgFace)
print(avgFace.shape)

# Compute eigenfaces on mean-subtracted training data
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T # (모든 얼굴) - (평균 얼굴에서 평균 열 벡터)
U, S, VT = np.linalg.svd(X,full_matrices=0) # 경제적 SVD 계산

# 약 2400개의 고유면을 가지고 있다.
fig1 = plt.figure()
ax1 = fig1.add_subplot(121)
img_avg = ax1.imshow(np.reshape(avgFace,(m,n)).T)
img_avg.set_cmap('gray')
plt.axis('off')

ax2 = fig1.add_subplot(122)
img_u1 = ax2.imshow(np.reshape(U[:,0],(m,n)).T)
img_u1.set_cmap('gray')
plt.axis('off')

plt.show()

* $ reshape face = x_{hat} = U_r * (U_r)^T * x $

In [ ]:
## Now show eigenface reconstruction of image that was omitted from test set
from matplotlib.image import imread

dog = True

if (not dog):
    testFace = faces[:,np.sum(nfaces[:36])] # First face of person 37, new face
    plt.imshow(np.reshape(testFace,(m,n)).T)
else:
    testDog = imread(os.path.join('..','DATA','dog.jpg'))
    testDog = np.mean(testDog, -1); # Convert RGB to grayscale
    m1, n1 = np.shape(testDog)
    testFace = np.zeros((m,n))

    for i in range(0, m):
        for j in range(0, n):            
            testFace[i, j] = testDog[i * int (m1/m), j * int (n1/n)]
    plt.imshow(testFace)
    testFace = np.reshape(testFace, m*n)


plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()


testFaceMS = testFace - avgFace
r_list = [25, 50, 100, 200, 400, 800, 1600] # rank에 따른 이미지 재구성

for r in r_list:

    reconFace = avgFace + U[:,:r]  @ U[:,:r].T @ testFaceMS #compute : 평균얼굴 + x_hat

    img = plt.imshow(np.reshape(reconFace,(m,n)).T)
    img.set_cmap('gray')
    plt.title('r = ' + str(r))
    plt.axis('off')
    plt.show()

# r값이 증가함에 따라 점점 실제 사진과 가까워지게 된다.
# 위상변화가 400모드 주변에서 발생하여 실제사람처럼 보이게 된다. >  따라서 400모드가 주어지면 실제 original image 사람을 선택할 수 있다.
# 중요한 의미 : 상대적으로 적은 양의 정보만으로도 선택할 수 있다.

In [ ]:
## Project person 2 and 7 onto PC5 and PC6 
# 라이브러리에서 무작위로 선택한 사람의 얼굴을 가져와서 투영.

P1num = 2 # Person number 2 
P2num = 7 # Person number 7 

P1 = faces[:,np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num])]
P2 = faces[:,np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num])]
# print(P1.shape)
# print(P2.shape)

P1 = P1 - np.tile(avgFace,(P1.shape[1],1)).T
P2 = P2 - np.tile(avgFace,(P2.shape[1],1)).T

PCAmodes = [5, 6] # Project onto PCA modes 5 and 6 #U행렬의 5,6번째 행렬을 선택
PCACoordsP1 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P1 
PCACoordsP2 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P2

plt.plot(PCACoordsP1[0,:],PCACoordsP1[1,:],'d',color='k',label='Person 2')
plt.plot(PCACoordsP2[0,:],PCACoordsP2[1,:],'^',color='r',label='Person 7')
plt.legend()
plt.show()

* 사람2 와 사람7이 주성분 공간에서 클러스터 되어있는것을 확인할 수 있다.
* plot이미지는 효율적으로 정확한 분류기를 구축할 수 있음을 알려줌.

In [ ]:
#일곱번째 사람의 이미지

img = plt.imshow(np.reshape(P2[:,0],(m,n)).T)
img.set_cmap('gray')
plt.axis('off')
plt.show()

In [ ]:
#두번째 사람의 이미지

img = plt.imshow(np.reshape(P1[:,0],(m,n)).T)
img.set_cmap('gray')
plt.axis('off')
plt.show()